In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-data-clean/Oct-2019.csv
/kaggle/input/amazon-sale-datasets-clean/Sales Amazon Cleaned.csv
/kaggle/input/amazon-sale-datasets-clean/Sales Amazon Cleaned_1.csv
/kaggle/input/amazon-sale-datasets-clean/Sales_Amazon_Cleaned_final.csv


In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/amazon-sale-datasets-clean/Sales_Amazon_Cleaned_final.csv")
df.head(5)

,Unnamed: 0,user_id,product_id,user_name,product_name,discounted_price,actual_price,discount_percentage,rating,rating_count,category_1,category_2,rating_score,difference_price,neg,neu,pos,compound
0,0,AG3D6O4STAQKAY2UVGEUV46KN35Q,B07JW9H4J1,Manav,Wayona Nylon Braided USB to Lightning Fast Cha...,399.0,1099.0,0.64,4.2,24269.0,Computers & Accessories,Accessories & Peripherals,Above Average,700.0,0.032,0.744,0.224,0.9033
1,1,AG3D6O4STAQKAY2UVGEUV46KN35Q,B07JW9H4J1,Manav,Wayona Nylon Braided USB to Lightning Fast Cha...,399.0,1099.0,0.64,4.2,24270.0,Computers & Accessories,Accessories & Peripherals,Above Average,700.0,0.032,0.744,0.224,0.9033
2,2,AG3D6O4STAQKAY2UVGEUV46KN35Q,B07JW9H4J1,Manav,Wayona Nylon Braided USB to Lightning Fast Cha...,399.0,1099.0,0.64,4.2,24269.0,Computers & Accessories,Accessories & Peripherals,Above Average,700.0,0.032,0.744,0.224,0.9033
3,3,AG3D6O4STAQKAY2UVGEUV46KN35Q,B07JW9H4J1,Adarsh gupta,Wayona Nylon Braided USB to Lightning Fast Cha...,399.0,1099.0,0.64,4.2,24269.0,Computers & Accessories,Accessories & Peripherals,Above Average,700.0,0.032,0.744,0.224,0.9033
4,4,AG3D6O4STAQKAY2UVGEUV46KN35Q,B07JW9H4J1,Adarsh gupta,Wayona Nylon Braided USB to Lightning Fast Cha...,399.0,1099.0,0.64,4.2,24270.0,Computers & Accessories,Accessories & Peripherals,Above Average,700.0,0.032,0.744,0.224,0.9033


# 1. Content-based Filtering

In [27]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from wordcloud import WordCloud
import time
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras 
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [28]:
users_cols = ["user_id_encoded", "user_name_encoded"]
rates_cols = ["user_id_encoded","product_id_encoded","rating","rating_count"]
items_cols = ["user_id_encoded","product_id","product_name","discounted_price","actual_price","discount_percentage","difference_price","neg","neu","pos","compound","category_1_encoded","category_2_encoded"]

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

items_cols = ["user_id_encoded","product_id_encoded","rating","discounted_price","actual_price","discount_percentage","difference_price","neg","neu","pos","compound","category_1_encoded","category_2_encoded"]

train_df, test_df = train_test_split(df, test_size=0.2, random_state = 42)  

label_encoder = LabelEncoder()

for column in train_df.select_dtypes(include=['object']).columns:
    train_df[column + '_encoded'] = label_encoder.fit_transform(train_df[column])

for column in test_df.select_dtypes(include=['object']).columns:
    test_df[column + '_encoded'] = label_encoder.fit_transform(test_df[column])

train_df = train_df[items_cols]

test_df = test_df[items_cols]

In [30]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [31]:
def CF_model():
    num_users = train_df['user_id_encoded'].nunique()
    num_products = train_df['product_id_encoded'].nunique()

    user_input = tf.keras.layers.Input(shape=(1,))
    product_input = tf.keras.layers.Input(shape=(1,))

    user_embedding = tf.keras.layers.Embedding(num_users, 10)(user_input)
    dense_user_1 = tf.keras.layers.Dense(32, activation='relu')(user_embedding)
    dense_user_2 = tf.keras.layers.Dense(16, activation='relu')(dense_user_1)



    product_embedding = tf.keras.layers.Embedding(num_products, 10)(product_input)
    dense_product_1 = tf.keras.layers.Dense(32, activation='relu')(product_embedding)
    dense_product_2 = tf.keras.layers.Dense(16, activation='relu')(dense_product_1)


    merged_embeddings = tf.keras.layers.Concatenate()([dense_user_2, dense_product_2])
    flatten_embeddings = tf.keras.layers.Flatten()(merged_embeddings)

    output = tf.keras.layers.Dense(1, activation='linear')(flatten_embeddings)

    model = tf.keras.models.Model(inputs=[user_input, product_input], outputs=output)
    
    return model
# build 
def CB_model():

    num_outputs = 32
    tf.random.set_seed(1)
    user_NN = tf.keras.models.Sequential([
        ### START CODE HERE ###   
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_outputs, activation='linear'),
        ### END CODE HERE ###  
    ])

    product_NN = tf.keras.models.Sequential([
        ### START CODE HERE ###     
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(num_outputs, activation='linear'),
        ### END CODE HERE ###  
    ])

    # create the user input and point to the base network
    input_user = tf.keras.layers.Input(shape=(1,))
    vu = user_NN(input_user)
    vu = tf.linalg.l2_normalize(vu, axis=1)

    # create the item input and point to the base network
    input_item = tf.keras.layers.Input(shape=(1,))
    vm = product_NN(input_item)
    vm = tf.linalg.l2_normalize(vm, axis=1)

    # compute the dot product of the two vectors vu and vm
    output = tf.keras.layers.Dot(axes=1)([vu, vm])


    model = tf.keras.models.Model(inputs=[input_user, input_item], outputs=output)

    return model
# model.summary()

In [32]:
def checkpoint_call():
    if not os.path.exists('checkpoints'):
        os.makedirs('checkpoints')
    model_ver = max([int(i) for i in os.listdir(f"checkpoints")]+[0]) + 1
    filepath=f"checkpoints/{model_ver}/"+"{epoch}.ckpt"
    return ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                                           save_weights_only=True, save_best_only=True, mode='min')

In [37]:
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100,
    decay_rate=0.96,
    staircase=True)

adam = optimizers.Adam(learning_rate=lr_schedule)

rmsprop = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)

optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

loss = tf.keras.losses.MeanSquaredError()

#Callback
logdir = 'logs'
tensorboard_callback = TensorBoard(log_dir=logdir)
checkpoint_callback = checkpoint_call()

In [41]:
models = {
   "CB_model":CB_model(),
    "CF_model": CF_model()
}

model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    rmsprop = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)

    model.compile(optimizer=rmsprop, loss=loss)
    
    h = model.fit([train_df['user_id_encoded'], train_df['user_id_encoded']], train_df['rating'],validation_split = 0.2,
#           validation_data = [[test_df['user_id_encoded'], test_df['user_id_encoded']], test_df['rating']], 
          epochs=3, batch_size=48, verbose=1)

    # Make predictions
    y_train_pred = model.predict([train_df['user_id_encoded'], train_df['user_id_encoded']])
    y_test_pred = model.predict([test_df['user_id_encoded'], test_df['user_id_encoded']])
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(train_df['rating'],y_train_pred )

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(test_df['rating'], y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Epoch 1/3
2025/2025 [==============================] - 11s 5ms/step - loss: 9.8272 - val_loss: 9.7936
Epoch 2/3
2025/2025 [==============================] - 9s 4ms/step - loss: 9.8148 - val_loss: 9.7933
Epoch 3/3
949/949 [==============================] - 2s 2ms/step
CB_model
Model performance for Training set
- Root Mean Squared Error: 3.1322
- Mean Absolute Error: 3.1221
- R2 Score: -155.3672
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 3.1342
- Mean Absolute Error: 3.1240
- R2 Score: -154.0931


Epoch 1/3
2025/2025 [==============================] - 16s 7ms/step - loss: 0.0811 - val_loss: 0.0074
Epoch 2/3
2025/2025 [==============================] - 13s 7ms/step - loss: 0.0058 - val_loss: 0.0023
Epoch 3/3
949/949 [==============================] - 3s 3ms/step
CF_model
Model performance for Training set
- Root Mean Squared Error: 0.0335
- Mean Absolute Error: 0.0097
- R2 Score: 0.9821
----------------------------------
Model performance

In [39]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
1,CF_model,-1.484640
0,CB_model,-154.093121


In [14]:
# hist = model.fit([train['user_index'], train['product_index']], train['interaction_score'],
#           validation_data = [[test['user_index'], test['product_index']], test['interaction_score']], 
#           epochs=3, batch_size=512, verbose=1)

In [ ]:
# loss = model.evaluate([test_df['user_id_encoded'], test_df['product_id_encoded']], test_df['rating'])
# print(f"Mean Squared Error on test data: {loss}")